In [29]:
import pandas as pd
from unidecode import unidecode
import numpy as np
import urllib.parse

In [70]:
import webbrowser

def remove_accents(input_str):
    return unidecode(input_str)

def search_and_display(csv_file, search_string):
    try:
        # Read data from the CSV file
        df = pd.read_csv(csv_file)

        # Prompt the user to select a search option
        while True:
            print("Select a search option:")
            print("1. Search in a specific language")
            print("2. Search across all languages")

            choice = input("Enter the number of your choice: ")

            if choice == '1':
                # Search in a specific language
                languages = df.columns.tolist()
                while True:
                    print("Select a language:")
                    for i, lang in enumerate(languages, 1):
                        print(f"{i}. {lang}")

                    lang_choice = input("Enter the number of your choice: ")

                    if not lang_choice.isdigit() or int(lang_choice) < 1 or int(lang_choice) > len(languages):
                        print("Invalid choice. Please select a valid language.")
                    else:
                        language = languages[int(lang_choice) - 1]
                        break

                # Remove accents from the search string
                search_string = remove_accents(search_string)

                # Remove accents from the data in the selected column
                df[language] = df[language].apply(remove_accents)

                # Perform the partial string match search
                result = df[df[language].str.contains(search_string, case=False, na=False)]

                if result.empty:
                    print("No matching rows found.")
                else:
                    print("Choose an option:")
                    options = result[language].tolist()
                    for i, option in enumerate(options, 1):
                        print(f"{i}. {option}")

                    choice = input("Enter the number of your choice: ")

                    if not choice.isdigit() or int(choice) < 1 or int(choice) > len(options):
                        print("Invalid choice.")
                    else:
                        choice = int(choice)
                        chosen_option = result.iloc[choice - 1]
                        print("\nChosen Option:")
                        print(f"{chosen_option[language]}\n")
                        
                        # Get the translations in other languages
                        translations = {}
                        for col in df.columns:
                            if col != language and col != 'en':  # Exclude the selected language and 'en' column
                                translations[col] = chosen_option[col]
                        
                        # Display the translations
                        print(f"{language}: {chosen_option[language]}")
                        for lang, translation in translations.items():
                            print(f"{lang}: {translation}")

                        # Ask if the user wants to Google search the chosen word
                        google_search_choice = input("Do you want to Google search this term? (yes/no): ").strip().lower()
                        if google_search_choice == 'yes':
                            google_search_url = f"https://www.google.com/search?q={chosen_option[language]}"

            elif choice == '2':
                # Search across all languages
                search_string = remove_accents(search_string)
                matching_words = []
                matching_languages = {}
                
                for col in df.columns:
                    col_matches = df[col][df[col].str.contains(search_string, case=False, na=False)].tolist()
                    if col_matches:
                        matching_words.extend(col_matches)
                        matching_languages[col] = col_matches

                if not matching_words:
                    print("No matching words found.")
                else:
                    print("Choose an option:")
                    for i, word in enumerate(matching_words, 1):
                        print(f"{i}. {word} ({', '.join([col for col, translations in matching_languages.items() if word in translations])})")

                    choice = input("Enter the number of your choice: ")

                    if not choice.isdigit() or int(choice) < 1 or int(choice) > len(matching_words):
                        print("Invalid choice.")
                    else:
                        choice = int(choice)
                        chosen_word = matching_words[choice - 1]
                        print("\nChosen Word:")
                        print(f"{chosen_word} ({', '.join([col for col, translations in matching_languages.items() if chosen_word in translations])})")

                        print("\nTranslations in Other Languages:")
                        for lang, translations in matching_languages.items():
                            if chosen_word in translations:
                                print(f"{lang}: {translations[translations.index(chosen_word)]}")

                        # Ask if the user wants to Google search the chosen word
                        google_search_choice = input("Do you want to Google search this term? (yes/no): ").strip().lower()
                        if google_search_choice == 'yes':
                            google_search_url = f"https://www.google.com/search?q={chosen_word}"

            else:
                print("Invalid choice. Please select a valid search option.")

            # Open the Google search URL if chosen
            if google_search_choice == 'yes':
                webbrowser.open(google_search_url)
                break

    except Exception as e:
        return f"An error occurred: {str(e)}"

In [71]:
csv_file = 'dictionary_data.csv'
search_string = input("Enter search string: ")
search_and_display(csv_file, search_string)

Enter search string:  math


Select a search option:
1. Search in a specific language
2. Search across all languages


Enter the number of your choice:  2


Choose an option:
1. Mathematische Analyse (de)
2. Mathematik (de)
3. Mathematical Analysis (en)
4. Mathematics (en)


Enter the number of your choice:  1



Chosen Word:
Mathematische Analyse (de)

Translations in Other Languages:
de: Mathematische Analyse


Do you want to Google search this term? (yes/no):  yes


In [68]:
def remove_accents(input_str):
    return unidecode(input_str)

def search_spanish_word(word):
    # Manually replace spaces with %20
    encoded_word = word.replace(' ', '%20')

    # Check if the word contains spaces
    if ' ' in word:
        # Word has spaces, construct URL for text translation
        query = '+'.join(encoded_word.split())
        english_url = f"https://en.pons.com/text-translation/spanish-english?q={query}"
        german_url = f"https://de.pons.com/übersetzung/spanisch-deutsch?q={query}"
    else:
        # Word is a single word, construct URL for translation
        english_url = f"https://en.pons.com/translate/spanish-english/{encoded_word}"
        german_url = f"https://de.pons.com/übersetzung/spanisch-deutsch/{encoded_word}"
    
    # Open both English and German translation pages in separate tabs
    webbrowser.open(english_url)
    webbrowser.open(german_url)

def search_english_word(word):
    
    encoded_word = word.replace(' ', '%20')
    if ' ' in word:
    # Manually replace spaces with '+'
        query = '+'.join(word.split())

    # Construct URLs for English-Spanish and English-German translations
        spanish_url = f"https://dictionary.cambridge.org/spellcheck/english-spanish/?q={query}"
        german_url = f"https://dictionary.cambridge.org/spellcheck/english-german/?q={query}"
    else:
        # Word is a single word, construct URL for translation
        english_url = f"https://dictionary.cambridge.org/spellcheck/english-spanish/{encoded_word}"
        german_url = f"https://dictionary.cambridge.org/spellcheck/english-spanish/{encoded_word}"
   
    # Open both Spanish and German translation pages in separate tabs
    webbrowser.open(spanish_url)
    webbrowser.open(german_url)

def search_and_display(csv_file, search_string):
    try:
        # Read data from the CSV file
        df = pd.read_csv(csv_file)

        while True:
            print("Select a search option:")
            print("1. Search in a specific language")
            print("2. Search across all languages")
            print("3. Exit")

            choice = input("Enter the number of your choice: ")

            if choice == '1':
                # Search in a specific language
                languages = df.columns.tolist()
                while True:
                    print("Select a language:")
                    for i, lang in enumerate(languages, 1):
                        print(f"{i}. {lang}")

                    lang_choice = input("Enter the number of your choice: ")

                    if not lang_choice.isdigit() or int(lang_choice) < 1 or int(lang_choice) > len(languages):
                        print("Invalid choice. Please select a valid language.")
                    else:
                        language = languages[int(lang_choice) - 1]
                        break

                # Remove accents from the search string
                search_string = remove_accents(search_string)

                # Remove accents from the data in the selected column
                df[language] = df[language].apply(remove_accents)

                # Perform the partial string match search
                result = df[df[language].str.contains(search_string, case=False, na=False)]

                if result.empty:
                    print("No matching rows found.")
                else:
                    print("Choose an option:")
                    options = result[language].tolist()
                    for i, option in enumerate(options, 1):
                        print(f"{i}. {option}")

                    choice = input("Enter the number of your choice: ")

                    if not choice.isdigit() or int(choice) < 1 or int(choice) > len(options):
                        print("Invalid choice.")
                    else:
                        choice = int(choice)
                        chosen_option = result.iloc[choice - 1]
                        print("\nChosen Option:")
                        print(f"{chosen_option[language]}\n")
                        for col in df.columns:
                            if col != language:
                                print(f"{col}: {chosen_option[col]}")

            elif choice == '2':
                # Search across all languages
                search_string = remove_accents(search_string)
                matching_words = []
                matching_languages = {}
                
                for col in df.columns:
                    col_matches = df[col][df[col].str.contains(search_string, case=False, na=False)].tolist()
                    if col_matches:
                        matching_words.extend(col_matches)
                        matching_languages[col] = col_matches

                if not matching_words:
                    print("No matching words found.")
                else:
                    print("Choose an option:")
                    for i, word in enumerate(matching_words, 1):
                        matching_languages_str = ", ".join([col for col, translations in matching_languages.items() if word in translations])
                        print(f"{i}. {word} ({matching_languages_str})")

                    choice = input("Enter the number of your choice: ")

                    if not choice.isdigit() or int(choice) < 1 or int(choice) > len(matching_words):
                        print("Invalid choice.")
                    else:
                        choice = int(choice)
                        chosen_word = matching_words[choice - 1]
                        print("\nChosen Word:")
                        print(f"{chosen_word} ({', '.join([col for col, translations in matching_languages.items() if chosen_word in translations])})")

                        print("\nTranslations in Other Languages:")
                        for lang, translations in matching_languages.items():
                            if chosen_word in translations:
                                print(f"{lang}: {translations[translations.index(chosen_word)]}")

                        google_search_choice = input("Do you want to search this word online? (yes/no): ").strip().lower()
                        if google_search_choice == 'yes':
                            if 'de' in matching_languages:
                                search_german_word(chosen_word)
                            elif 'es' in matching_languages:
                                search_spanish_word(chosen_word)
                            elif 'en' in matching_languages:
                                search_english_word(chosen_word)

            elif choice == '3':
                print("Exiting...")
                break

            else:
                print("Invalid choice. Please select a valid search option.")

    except Exception as e:
        return f"An error occurred: {str(e)}"
